In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 7.2MB/s 
     |████████████████████████████████| 2.9MB 35.6MB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=222bad32783cb0db8f8d603f7d7c64a5682a0ec7e5ab10a032a4ae40babe88da
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 133.6MB 102kB/s 
     |████████████████████████████████| 61kB 2.8MB/s 
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [ ]:
import numpy as np
import pandas as pd
import transformers
import ast

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/SQUAD files/train.csv')
df_valid = pd.read_csv('/content/drive/MyDrive/SQUAD files/valid.csv')

In [ ]:
train_input_ids = np.load('/content/drive/MyDrive/SQUAD files/bert_train_input_ids.npy')
train_token_type_ids = np.load('/content/drive/MyDrive/SQUAD files/bert_train_token_type_ids.npy')
train_attention_mask = np.load('/content/drive/MyDrive/SQUAD files/bert_train_attention_mask.npy')

In [ ]:
valid_input_ids = np.load('/content/drive/MyDrive/SQUAD files/bert_valid_input_ids.npy')
valid_token_type_ids = np.load('/content/drive/MyDrive/SQUAD files/bert_valid_token_type_ids.npy')
valid_attention_mask = np.load('/content/drive/MyDrive/SQUAD files/bert_valid_attention_mask.npy')

# Document Relevance Discriminator.

In [ ]:
# imports pytorch
import torch

import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = xm.xla_device()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class squad_dataset(Dataset):

    def __init__(self, input_ids, attention_mask, token_type_ids, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.labels = labels
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = {'input_ids': self.input_ids[idx],
                  'attention_mask': self.attention_mask[idx],
                  'token_type_ids':self.token_type_ids[idx],
                  'labels':self.labels[idx]}
        
        return sample

In [ ]:
BATCH_SIZE=10

In [ ]:
train_dataset = squad_dataset(train_input_ids, train_attention_mask, train_token_type_ids, df_train['label'].values)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
dev_dataset = squad_dataset(valid_input_ids, valid_attention_mask, valid_token_type_ids, df_valid['label'].values)
dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
# Parameters:
lr = 3e-5
max_grad_norm = 1.0
num_training_steps = int(len(df_train) / BATCH_SIZE * epochs)
num_warmup_steps = 5213
warmup_proportion = float(num_warmup_steps) / float(num_training_steps)  # 0.1

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

# Store our loss and accuracy for plotting
train_loss_set = []

In [ ]:
for epoch in range(epochs):
    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    running_loss = 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        # batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch['input_ids'], batch['attention_mask'], batch['token_type_ids'], batch['labels']

        ###############Bug fix code####################
        b_input_ids = b_input_ids.type(torch.LongTensor)
        b_input_mask = b_input_mask.type(torch.LongTensor)
        b_token_type_ids = b_token_type_ids.type(torch.LongTensor)
        b_labels = b_labels.type(torch.LongTensor)

        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_token_type_ids = b_token_type_ids.to(device)
        b_labels = b_labels.to(device)
         ############################################
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids = b_input_ids, attention_mask=b_input_mask, token_type_ids=b_token_type_ids, labels=b_labels)
        loss, logits = outputs[:2]

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
        optimizer.step()
        scheduler.step()
        xm.mark_step()
        
        running_loss = loss.item()

        if step%2000 == 1999:
            print(f'Epoch:{epoch} Batch:{step} Loss:{running_loss}')
            running_loss = 0
    
print('Training finished.')

Epoch:0 Batch:1999 Loss:0.3447030484676361
Epoch:0 Batch:3999 Loss:0.8385086059570312
Epoch:0 Batch:5999 Loss:0.6147722601890564
Epoch:0 Batch:7999 Loss:0.421488493680954
Epoch:0 Batch:9999 Loss:0.2366858571767807
Epoch:0 Batch:11999 Loss:0.540687084197998
Epoch:1 Batch:1999 Loss:0.12257383018732071
Epoch:1 Batch:3999 Loss:0.2126931995153427
Epoch:1 Batch:5999 Loss:0.07396338135004044
Epoch:1 Batch:7999 Loss:0.05300985649228096
Epoch:1 Batch:9999 Loss:0.1040070429444313
Epoch:1 Batch:11999 Loss:0.0651516243815422
Epoch:2 Batch:1999 Loss:0.010127833113074303
Epoch:2 Batch:3999 Loss:1.0037075281143188
Epoch:2 Batch:5999 Loss:0.4034174382686615
Epoch:2 Batch:7999 Loss:0.06033485755324364
Epoch:2 Batch:9999 Loss:0.0007344763143919408
Epoch:2 Batch:11999 Loss:0.005358055699616671
Epoch:3 Batch:1999 Loss:0.0012756527867168188
Epoch:3 Batch:3999 Loss:0.4504532814025879
Epoch:3 Batch:5999 Loss:0.0013328526401892304
Epoch:3 Batch:7999 Loss:0.00044214120134711266
Epoch:3 Batch:9999 Loss:0.000500

In [ ]:
model.save_pretrained('/content/drive/MyDrive/DRD/')

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, batch in enumerate(dev_dataloader):

            b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch['input_ids'], batch['attention_mask'], batch['token_type_ids'], batch['labels']

            ###############Bug fix code####################
            b_input_ids = b_input_ids.type(torch.LongTensor)
            b_input_mask = b_input_mask.type(torch.LongTensor)
            b_token_type_ids = b_token_type_ids.type(torch.LongTensor)
            b_labels = b_labels.type(torch.LongTensor)

            b_input_ids = b_input_ids.to(device)
            b_input_mask = b_input_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_labels = b_labels.to(device)
            
            outputs = model(b_input_ids, b_input_mask, b_token_type_ids)
            fin_targets.extend(b_labels.cpu().detach().numpy().tolist())
            outputs = outputs.logits
            fin_outputs.extend(outputs.cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
for epoch in range(1):
    outputs, targets = validation(epoch)

In [ ]:
outputs_11 = np.argmax(outputs, axis=1)

In [ ]:
accuracy = accuracy_score(targets, outputs_11)
print(f'Validation accuracy is {accuracy}.')

Validation accuracy is 0.7570959319464331.


# Answer Verifier Discriminiator.

### Preparing RACE dataset.

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/RACE files/race_train_data.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/RACE files/race_dev_data.csv')
test_df = pd.read_csv('/content/drive/MyDrive/RACE files/race_test_data.csv')

In [ ]:
train_df.head()

,answerKey,article,question,options
0,B,Do you like painting? Have you ever seen a pai...,What special things can you see on a road in S...,"['Some colourful trees.', 'Paintings in tree h..."
1,A,Do you like painting? Have you ever seen a pai...,Wang Yue paints the tree hole paintings to .,"['make the ugly tree holes beautiful', 'make a..."
2,C,Do you like painting? Have you ever seen a pai...,What does Wang Yue worry about before she begi...,"['The cost she will spend.', 'The thoughts fro..."
3,B,Do you like painting? Have you ever seen a pai...,What does Wang Yue think of painting on a tree...,"[""It's a boring job."", ""It's a hard job."", ""It..."
4,D,Do you like painting? Have you ever seen a pai...,What's the best title for this passage?,"['Trees in Shijiazhuang', 'Making Our Roads Be..."


For a given datapoint, we will divide it into number of options available.

Thus if we have 4 options then we will get 4 total points from that and if we have 5 points then we will have 5 points.

In [ ]:
train_df['article'].iloc[0]

'Do you like painting? Have you ever seen a painting in a tree hole? Now you can see many paintings in tree holes on a road in Shijiazhuang, Hebei Province. What\'s up?\nThese paintings are painted by Wang Yue, 23, a college student at Dalian Polytechnic University. Wang describes her tree hole paintings as "a beautiful journey" through which she adds fun to her hometown.\n"When I saw that the trees by the roadside had lost all their leaves, the ugly tree holes became obvious ," she said.\n"I thought if I could paint some interesting scenes in these holes they might add some color to the city. "\nBut Wang was worried that the pigments  in her watercolors could hurt the trees. After asking environmental specialists and getting permission from city officials, Wang gathered her color palette , brushes and other things to get to work.\n"Painting on a tree hole is not as easy as painting on a canvas . I had to think about the shapes and sizes of the tree holes and adjust my designs accordin

In [ ]:
label_map = {'A':0,
             'B':1,
             'C':2,
             'D':3}

In [ ]:
def race_construct(dataset):

    labels = []
    options = []
    contexts = []
    questions = []

    for i in range(len(dataset)):
        context = dataset['article'].iloc[i]
        question = dataset['question'].iloc[i]
        options_list = ast.literal_eval(dataset['options'].iloc[i])
        answer = label_map.get(dataset['answerKey'].iloc[i])

        for j in range(len(options_list)):
            if j == answer:
                label = 1
            else:
                label = 0
            labels.append(label)
            contexts.append(context)
            questions.append(question)
            options.append(options_list[j])
    
    return questions, options, contexts, labels

In [ ]:
questions, options, contexts, labels = race_construct(train_df)

In [ ]:
new_train_df = pd.DataFrame()

new_train_df['question'] = questions
new_train_df['option'] = options
new_train_df['context'] = contexts
new_train_df['label'] = labels

In [ ]:
new_train_df.head()

,question,option,context,label
0,What special things can you see on a road in S...,Some colourful trees.,Do you like painting? Have you ever seen a pai...,0
1,What special things can you see on a road in S...,Paintings in tree holes.,Do you like painting? Have you ever seen a pai...,1
2,What special things can you see on a road in S...,Many posters on destroyed trees.,Do you like painting? Have you ever seen a pai...,0
3,What special things can you see on a road in S...,Some ads on both sides of the roads.,Do you like painting? Have you ever seen a pai...,0
4,Wang Yue paints the tree hole paintings to .,make the ugly tree holes beautiful,Do you like painting? Have you ever seen a pai...,1


In [ ]:
#Checking that new dataframe is 4 times the size of previous one.
len(train_df), len(new_train_df) 

(87897, 351588)

In [ ]:
questions, options, contexts, labels = race_construct(dev_df)

new_dev_df = pd.DataFrame()

new_dev_df['question'] = questions
new_dev_df['option'] = options
new_dev_df['context'] = contexts
new_dev_df['label'] = labels

In [ ]:
#Checking that new dataframe is 4 times the size of previous one.
len(dev_df), len(new_dev_df)

(4887, 19548)

In [ ]:
questions, options, contexts, labels = race_construct(test_df)

new_test_df = pd.DataFrame()

new_test_df['question'] = questions
new_test_df['option'] = options
new_test_df['context'] = contexts
new_test_df['label'] = labels

In [ ]:
#Checking that new dataframe is 4 times the size of previous one.
len(test_df), len(new_test_df)

(4934, 19736)

In [ ]:
new_train_df.to_csv('/content/drive/MyDrive/RACE files/race_train_df_ard.csv', index=False)
new_dev_df.to_csv('/content/drive/MyDrive/RACE files/dev_df_ard.csv', index=False)
new_test_df.to_csv('/content/drive/MyDrive/RACE files/test_df_ard.csv', index=False)

### Preparing Input ids.

In [ ]:
new_train_df = pd.read_csv('/content/drive/MyDrive/RACE files/race_train_df_ard.csv')
new_dev_df = pd.read_csv('/content/drive/MyDrive/RACE files/dev_df_ard.csv')

Now as we know number of samples with label 0 are 3 times that of number of samples with label +1. Now due to this there's data imbalance.

So we have to reduce the number of samples with label -1.

In [ ]:
MAX_LEN = 512 #Taking 512 because context are pretty lengthy.

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def race_preprocessor(dataset, tokenizer):
    '''
    This function will convert a given article, question, choices in a format:

    [CLS] question [SEP] answer [SEP] context [SEP]
    [CLS] question [SEP] answer [SEP] context [SEP]
    [CLS] question [SEP] answer [SEP] context [SEP]
    [CLS] question [SEP] answer [SEP] context [SEP]

    After converting in this format the data will be tokenized using a given tokenizer.
    This function will return 4 arrays namely, input_ids, attention_mask, token_type_ids and labels.

    individual input_ids, token_type_ids, attention_mask shape will be as: [num_choices, max_seq_length]
    '''
    all_input_ids = []
    all_attention_mask = []

    for i in range(len(dataset)):

        article = dataset['context'].iloc[i]
        question = dataset['question'].iloc[i]
        option = dataset['option'].iloc[i]

        input_string = '[CLS] ' + question + ' [SEP] ' + option + ' [SEP] ' + article + ' [SEP]'

        input_ids = tokenizer.encode(input_string, max_length=MAX_LEN, truncation='longest_first', add_special_tokens=False)

        attention_mask = [1]*len(input_ids)

        padding_length = MAX_LEN - len(input_ids)

        input_ids = input_ids + [0]*padding_length
        attention_mask = attention_mask + [0]*padding_length

        assert len(input_ids) == MAX_LEN
        assert len(attention_mask) == MAX_LEN

        all_input_ids.append(np.asarray(input_ids, dtype='int32'))
        all_attention_mask.append(np.asarray(attention_mask, dtype='int32'))

    return all_input_ids, all_attention_mask

In [ ]:
ard_train_input_ids, ard_train_attention_mask = race_preprocessor(new_train_df, tokenizer)

In [ ]:
ard_dev_input_ids, ard_dev_attention_mask = race_preprocessor(new_dev_df, tokenizer)

In [ ]:
ard_test_input_ids, ard_test_attention_mask = race_preprocessor(new_test_df, tokenizer)

In [ ]:
ard_train_input_ids = np.asarray(ard_train_input_ids, dtype='int32')
ard_train_attention_mask = np.asarray(ard_train_attention_mask, dtype='int32')

In [ ]:
ard_dev_input_ids = np.asarray(ard_dev_input_ids, dtype='int32')
ard_dev_attention_mask = np.asarray(ard_dev_attention_mask, dtype='int32')

In [ ]:
ard_test_input_ids = np.asarray(ard_test_input_ids, dtype='int32')
ard_test_attention_mask = np.asarray(ard_test_attention_mask, dtype='int32')

In [ ]:
np.save('/content/drive/MyDrive/RACE files/ard_train_input_ids', ard_train_input_ids)
np.save('/content/drive/MyDrive/RACE files/ard_train_attention_mask', ard_train_attention_mask)

In [ ]:
np.save('/content/drive/MyDrive/RACE files/ard_dev_input_ids', ard_dev_input_ids)
np.save('/content/drive/MyDrive/RACE files/ard_dev_attention_mask', ard_dev_attention_mask)

In [ ]:
np.save('/content/drive/MyDrive/RACE files/ard_test_input_ids', ard_test_input_ids)
np.save('/content/drive/MyDrive/RACE files/ard_test_attention_mask', ard_test_attention_mask)

### Training the Sequence Classification Model on the data.

In [ ]:
import torch

import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = xm.xla_device()

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/RACE files/race_train_df_ard.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/RACE files/dev_df_ard.csv')

In [ ]:
train_input_ids = np.load('/content/drive/MyDrive/RACE files/ard_train_input_ids.npy')
train_attention_mask = np.load('/content/drive/MyDrive/RACE files/ard_train_attention_mask.npy')

In [ ]:
dev_input_ids = np.load('/content/drive/MyDrive/RACE files/ard_dev_input_ids.npy')
dev_attention_mask = np.load('/content/drive/MyDrive/RACE files/ard_dev_attention_mask.npy')

In [ ]:
class race_dataset(Dataset):

    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = {'input_ids': self.input_ids[idx],
                  'attention_mask': self.attention_mask[idx],
                  'labels':self.labels[idx]}
        
        return sample

In [ ]:
BATCH_SIZE=10

In [ ]:
train_dataset = race_dataset(train_input_ids, train_attention_mask, train_df['label'].values)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
dev_dataset = race_dataset(dev_input_ids, dev_attention_mask, dev_df['label'].values)
dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

In [ ]:
int(len(train_df) / BATCH_SIZE * 2)

70317

In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
# Parameters:
lr = 5e-5
max_grad_norm = 1.0
num_training_steps = int(len(train_df) / BATCH_SIZE * epochs)
num_warmup_steps = 7031
warmup_proportion = float(num_warmup_steps) / float(num_training_steps)  # 0.1

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

# Store our loss and accuracy for plotting
train_loss_set = []

In [ ]:
for epoch in range(epochs):
    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    running_loss = 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        # batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']

        ###############Bug fix code####################
        b_input_ids = b_input_ids.type(torch.LongTensor)
        b_input_mask = b_input_mask.type(torch.LongTensor)
        b_labels = b_labels.type(torch.LongTensor)

        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_labels = b_labels.to(device)
         ############################################
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids = b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss, logits = outputs[:2]

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
        optimizer.step()
        scheduler.step()
        xm.mark_step()
        
        running_loss = loss.item()

        if step%2000 == 1999:
            print(f'Epoch:{epoch} Batch:{step} Loss:{running_loss}')
            running_loss = 0
    
print('Training finished.')

Epoch:0 Batch:1999 Loss:0.5961182713508606
Epoch:0 Batch:3999 Loss:0.41820284724235535
Epoch:0 Batch:5999 Loss:0.8303608298301697
Epoch:0 Batch:7999 Loss:0.5166329145431519
Epoch:0 Batch:9999 Loss:0.8408998847007751
Epoch:0 Batch:11999 Loss:0.3997719883918762
Epoch:0 Batch:13999 Loss:0.6077042818069458
Epoch:0 Batch:15999 Loss:0.6269365549087524
Epoch:0 Batch:17999 Loss:0.8068832755088806
Epoch:0 Batch:19999 Loss:0.6264435648918152
Epoch:0 Batch:21999 Loss:0.8706437945365906
Epoch:0 Batch:23999 Loss:0.5016117095947266
Epoch:0 Batch:25999 Loss:0.6213876008987427
Epoch:0 Batch:27999 Loss:0.7354616522789001
Epoch:0 Batch:29999 Loss:0.39805570244789124
Epoch:0 Batch:31999 Loss:0.3023308217525482
Epoch:0 Batch:33999 Loss:0.5222350358963013
Epoch:1 Batch:1999 Loss:0.40528154373168945
Epoch:1 Batch:3999 Loss:0.6116301417350769
Epoch:1 Batch:5999 Loss:0.40735387802124023
Epoch:1 Batch:7999 Loss:0.496126651763916
Epoch:1 Batch:9999 Loss:0.6106532216072083
Epoch:1 Batch:11999 Loss:0.509648740291

In [ ]:
model.save_pretrained('/content/drive/MyDrive/ARD/')

### Inference.

In [ ]:
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/ARD/')

In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, batch in enumerate(dev_dataloader):

            b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']

            ###############Bug fix code####################
            b_input_ids = b_input_ids.type(torch.LongTensor)
            b_input_mask = b_input_mask.type(torch.LongTensor)
            b_labels = b_labels.type(torch.LongTensor)

            b_input_ids = b_input_ids.to(device)
            b_input_mask = b_input_mask.to(device)
            b_labels = b_labels.to(device)
            
            outputs = model(b_input_ids, b_input_mask)
            fin_targets.extend(b_labels.cpu().detach().numpy().tolist())
            outputs = outputs.logits
            fin_outputs.extend(outputs.cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
for epoch in range(1):
    outputs, targets = validation(epoch)

In [ ]:
outputs_11 = np.argmax(outputs, axis=1)

In [ ]:
accuracy = accuracy_score(targets, outputs_11)
print(f'Validation accuracy is {accuracy}.')

Validation accuracy is 0.75.
